In [1]:
# Imports
import plotly.express as px
from helper import *
import pandas as pd
import numpy as np
import gzip
import json
from scipy import stats

# Trust and mobility 
### mobility

During the pandemic, people were engaging in a variety of activities in response to the spread of COVID-19. As the pandemic unfolded, many people turned to Wikipedia and other sources of information to learn more about the virus and the measures being taken to combat it. Some people may have started going out less in order to reduce the risk of transmission, while others may have continued to go out as usual, either out of a sense of trust in their government's handling of the crisis or for other reasons. Let's have a look at how mobility was affected by the covid-19 wave. 




In [2]:
# Imports
import plotly.express as px
from helper import *
import pandas as pd
import numpy as np
import gzip
import json
from scipy import stats

Here we extract the mobility of all countries and separate them into moving category: which accounts for movement in Retail and Recreations, Parks,Transit stations and Workplace. And the covid category: which accounts for momement in Grocery and Pharmacy and Residential. The metric is a change in mobility from baseline in percentage, meaning that positive values represent more mobiltiy than normal and negative representing the inverse, with the baseline being before covid-19 happened. 

We will focus on the moving category and calculate the median of all countries per day.




In [3]:
#Load mobility data 

mobility = get_mobility_df(get_country_dict('trust gov mobility'))
moving_mobility_df = mobility['moving category'].reset_index()

# Select countries that appear in both the trust and mobility dataset
moving_mobility_df['country_region'] =moving_mobility_df['country_region'].map(get_country_dict('trust gov mobility'))

# Calculate median moving mobility on each day and save in a dataframe
median_mobility_df = moving_mobility_df.groupby('date')['moving category'].median().to_frame().reset_index()
median_mobility_df = median_mobility_df.rename(columns={'moving category':'Median mobility change from baseline', 'date': 'Date'})

/Users/charlottsertic/Documents/GitHub/ada-2022-project-thedatadiggers22/helper.py:446: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  google_mobility = pd.read_csv(data_folder+'Global_Mobility_Report.csv.zip')


Let's plot the timeseries of the median mobility over the six month period 

In [4]:
df = px.data.stocks()
fig = px.line(median_mobility_df, x='Date', y='Median mobility change from baseline', title='Median mobility change from baseline over time<br><sup>The median mobility change from baseline was calculated over 30 countries</sup>')
fig.show()


We can see that there is a sharp drop in mobility for all countries in the middle of march, exactly when there were all the lockdowns in europe mid march 2020. Then we can see the curve slowly increased until june, and then there is an above baseline moving mobility, meaning that people were going out more than usual. This can be because people had enough of staying inside and had been deprived of social events for months, so they felt the need to go out more and be more social. This can also be due to it being summer, where people natrually go out more and are more social.


### Mobility and death and pageviews

Now let's analyse the changes in mobility over the 6 month period with respect to covid deaths and pageviews, to see how these are related. We also introduce the Trust dataset which contains information on how much each country trusts their goverment, in a percentage. To visualize this we create a bubble scatter plot.


Load Pageviews, Death and Trust

In [12]:
# Load and preprocess pageviews and deaths 
pageview_df = pd.read_csv("page_views_covid_related.csv.gz")
population_df = pd.read_csv("Population_countries.csv")

_,_, df_pageviews100k,_ = get_pageviews_df(pageview_df, population_df, get_country_dict('trust gov mobility'), '2020-01-22', '2020-11-22')
_, _, _, _, deaths100k, _, _,_ = get_cases_deaths_df(population_df, get_country_dict('trust gov mobility'), '2020-01-22', '2020-11-22')



# Transpose datagrames and rename columns
df_pageviews100k = df_pageviews100k.transpose().stack().to_frame().reset_index().rename(columns={"level_0": "country", "date": "date", 0:"pageviews per 100k"}, errors="raise")
deaths100k = deaths100k.transpose().stack().to_frame().reset_index().rename(columns={"level_0": "country", "date": "date", 0:"deaths per 100k"}, errors="raise")

# Merge pageviews and deaths onto df_animation which will contain columns : ['Month-Year','Country',	'Cumulative pageviews per 100k','Cumulative deaths per 100k','Mobility change from baseline','Trust']
df_animation = df_pageviews100k.merge(deaths100k, on=['country','date'])
df_animation['date'] = pd.to_datetime(df_animation['date'])
df_animation=df_animation.rename(columns={"country": "Country"})

In [6]:
# Dowload and preprocess trust dataset
data_folder = 'data_2/'
df_trust_gov = pd.read_csv(data_folder+'share-who-trust-government.csv.zip') 
df_trust_gov = df_trust_gov.set_index("Entity")[["Trust the national government in this country"]].transpose()[COUNTRY_OWN_LANG_TRUST_GOV.keys()].rename(columns= COUNTRY_OWN_LANG_TRUST_GOV)
country_dict = get_country_dict('trust gov mobility')

# # Map the trust category to the countries in df_animation
trust_dict =df_trust_gov.to_dict('index')
df_animation['Trust'] = df_animation['Country'].map(trust_dict['Trust the national government in this country'])
df_animation.head()

,Country,date,pageviews per 100k,deaths per 100k,Trust
0,it,2020-01-22,40.388883,0.0,52.3
1,it,2020-01-23,46.878422,0.0,52.3
2,it,2020-01-24,38.747154,0.0,52.3
3,it,2020-01-25,54.543748,0.0,52.3
4,it,2020-01-26,55.928115,0.0,52.3


In [7]:
#Function
def f(x):
    d = {}
    d['Cumulative pageviews per 100k'] = x['pageviews per 100k'].sum()
    d['Cumulative deaths per 100k'] = x['deaths per 100k'].sum()
    d["Mobility change from baseline"] = x['moving category'].mean()


    return pd.Series(d, index=['Cumulative pageviews per 100k','Cumulative deaths per 100k', "Mobility change from baseline"])
    
# Add mobility to df_animation
# merge dataframe with the mobility dataset
df_animation = df_animation.merge(moving_mobility_df, left_on=['Country','date'],  right_on=['country_region','date'])
df_animation = df_animation.drop(columns=['country_region','country_region_code'])
df_animation['date']=df_animation['date'].astype(str)

# Rename the date to make it clearer
df_animation['Month-Year'] = pd.to_datetime(df_animation['date']).dt.to_period('M') 
df_animation['Month-Year']=df_animation['Month-Year'].astype(str)

# Rename the country codes to country anmes
inv_map = {v: k for k, v in get_country_dict('trust gov mobility').items()}
df_animation['Country'] = df_animation['Country'].map(inv_map)

# Apply funciton in helper to apply aggregate functions 
grouped_df = df_animation.groupby(['Month-Year','Country']).apply(f)
grouped_df = grouped_df.reset_index()
grouped_df =grouped_df.merge(df_animation[['Country','Month-Year','Trust']], how='left', on=['Month-Year','Country']).drop_duplicates()

# Change trust to value between 0 and 1
grouped_df["Trust"] = grouped_df["Trust"]/100

df = grouped_df.round(decimals=3) # Round dataset values to 3 decimal places

grouped_df.head()

,Month-Year,Country,Cumulative pageviews per 100k,Cumulative deaths per 100k,Mobility change from baseline,Trust
0,2020-02,Bangladesh,0.556806,0.0,5.250000,0.830
15,2020-02,Bulgaria,161.407208,0.0,5.097669,0.404
30,2020-02,Croatia,171.629180,0.0,8.503883,0.420
45,2020-02,Czechia,1925.544347,0.0,6.410000,0.458
60,2020-02,Denmark,275.594008,0.0,-1.578199,0.793


Let's plot the bubble scatter plot:

In [8]:
fig = px.scatter(df,x="Mobility change from baseline", y="Cumulative deaths per 100k" , animation_frame="Month-Year", animation_group="Country",
           size="Cumulative pageviews per 100k" ,color="Trust", hover_name="Country", range_x=[-80,60],range_y=[-20,40], size_max=80, template='plotly_dark' )
      
fig.update_layout(height=500, width= 900,title_text="Cumulative deaths per 100k vs. Mobility change from baseline, 2020")
fig.update_layout(margin=dict(l=40, r=10, t=125, b=10))  
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.28,showarrow=False,text = "The x-axis shows the mean mobility change from baseline per month.")
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.22,showarrow=False,text = "The y-axis shows the cumulative deaths per 100k per month.")
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.16,showarrow=False,text = "The size of the bubble shows the cumulative pageviews per 100k per month ")
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.10,showarrow=False,text = "The colour of the bubble corresponds to how much people trust their government.")
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.show()

fig.write_html('data_2/Images/bubble_plot.html')


We can clearly see the bubbles moving up and expanding as they move to the left. This means that as there are more covid related deaths there are more covid related pageviews and less mobility overall. We also see the reverse effect as we reach july. Moreover, we see that in the months of March and April, countries that are less government trusting have a lower mobility change from baseline, therefore they are going out less. Let's explore this in further detail.

### Mobility and trust 

Let's look at mobility and Trust. For mobility we take the minimium mobility value, as this was where we saw most disparites in the plot above. 

In [9]:
# Create mobility_trust dataframe 
mobility_trust = mobility['moving category'].reset_index()
mobility_trust['country_region'] =mobility_trust['country_region'].map(get_country_dict('trust gov mobility'))
mobility_trust = mobility_trust.groupby('country_region')['moving category'].min().to_frame()
mobility_trust = mobility_trust.reset_index()
mobility_trust['Trust the national government in this country'] = mobility_trust['country_region'].map(trust_dict['Trust the national government in this country'])
mobility_trust['country_region'] = mobility_trust['country_region'].map(inv_map)
mobility_trust = mobility_trust.rename(columns={"moving category": "Min mobility change from baseline","country_region": "Country"})
mobility_trust.head()

,Country,Min mobility change from baseline,Trust the national government in this country
0,Bulgaria,-62.327553,40.4
1,Bangladesh,-61.750000,83.0
2,Czechia,-65.633333,45.8
3,Denmark,-35.839475,79.3
4,Germany,-56.426471,82.0


In [10]:
fig = px.scatter(mobility_trust, x='Trust the national government in this country', y="Min mobility change from baseline",text="Country", trendline="ols", title='Relationship between min moblitiy change from baseline and trust in the national government in a country, 2020')

# fig.update_layout(height=800,title_text='Relationship between min moblitiy change from baseline and trust in the national government in a country')
fig.update_traces(textposition="bottom right")
fig.update_layout(margin=dict(l=20, r=20, t=100, b=0))  
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.13,showarrow=False,text = "The x-axis the trust in the national government in a country.")
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.09,showarrow=False,text = "The y-axis shows the min mobility change from baseline per month.")
fig.update_layout(yaxis_ticksuffix = "%")
fig.update_layout(xaxis_ticksuffix = "%")
fig.show()



Linear Regression 

Above we have plotted the regression line of trust in the national governent and min mobility change.

In [11]:
# Linear regression summary
result = px.get_trendline_results(fig)
result_pval = result.iloc[0]['px_fit_results'].pvalues
result_coeff = result.iloc[0]['px_fit_results'].params
print('regression summary: ')
print('Intercept: {}, p-value: {}'.format(result_coeff[0],result_pval[0]))
print('Coefficent: {}, p-value: {}'.format(result_coeff[1],result_pval[1]))

regression summary: 
Intercept: -89.95318296260504, p-value: 8.413095308649931e-10
Coefficent: 0.48095989431734393, p-value: 0.007790561016220843


Interpretation:

We have an intercept of -90%, with a p value of order -10 which means that it is statistically significant. We can thus say that hypothetically we would expect a country with 0% trust to have a minimum mobility of -90%. 
Moreover, looking at the coefficient we have 0.481, which is also statistically signifiant. This means that that an increase trust in the government by 10% will imply an increase of 5% of minimum mobility change. 

Therefore we see that government-trusting populations have a higher minimum mobility change, meaning that trusting your government meant that you did not alter and restrict your mobility as much. This could be because a trusting government instills confidence and people are therefore less scared of covid and are able to maintain a lifestyle closer to the norm.